# <h1 align=center> PROYECTO INDIVIDUAL Nº1 </h1>
<hr>
<h2 align=center>Extracción, transformación y carga de los datos (ETL)</h2>

**Importamos librerias.-**

In [22]:

import pandas as pd
import numpy as np
import  sqlalchemy as sql


**1-Extracción de los datos**

Lo primero que hacemos es leer los datos de los archivos .csv y json utilizando dataframe de pandas.

In [23]:

df_amazon = pd.read_csv("../Datasets/amazon_prime_titles.csv", index_col=0 )
df_disney = pd.read_csv("../Datasets/disney_plus_titles.csv", index_col=0  )
df_hulu = pd.read_csv("../Datasets/Hulu_titles.csv", index_col=0)
df_netflix = pd.read_json("../Datasets/netflix_titles.json")

Establecemos "show_id" como indice.

In [24]:
df_netflix.set_index("show_id", inplace=True)
df_netflix.head(3)

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
show_id,,,,,,,,,,,
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...


**2.Transformaciones.-**

Agregamos una columna con el nombre de la plataforma que permitira diferenciar el origen de los datos cuando sean concatenados.

In [25]:
df_amazon["platform"] = "amazon"
df_disney["platform"] = "disney"
df_hulu["platform"] = "hulu"
df_netflix["platform"] = "netflix"


Copiamos los valores que contienen  "min" o "Season" del dataframe de hulu y de netflix de la columna "rating" a la columna "duration", siempre y cuando esta se encuentre vacia.

In [26]:
filtro_hulu =( df_hulu["rating"].str.contains("min")| df_hulu["rating"].str.contains("Season") & df_hulu["duration"].isnull())

In [27]:
df_hulu.loc[filtro_hulu,'duration']= df_hulu.loc[filtro_hulu, 'rating']

In [28]:
df_netflix['rating'].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', None,
       'TV-Y7-FV', 'UR'], dtype=object)

In [29]:
filtro_netflix =( df_netflix["rating"].str.contains("min")| df_netflix["rating"].str.contains("Season") & df_netflix["duration"].isnull())

In [30]:
df_netflix.loc[filtro_netflix,'duration']= df_netflix.loc[filtro_netflix, 'rating']

Eliminamos los valores erroneos.

In [31]:

df_hulu.loc[filtro_hulu,"rating"] = None

In [32]:
df_netflix.loc[filtro_netflix,"rating"] = None

Concatenamos los dataframes

In [33]:
df = pd.concat([df_amazon, df_disney, df_hulu, df_netflix], ignore_index=True)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          22998 non-null  object
 1   title         22998 non-null  object
 2   director      14739 non-null  object
 3   cast          17677 non-null  object
 4   country       11499 non-null  object
 5   date_added    13444 non-null  object
 6   release_year  22998 non-null  int64 
 7   rating        21839 non-null  object
 8   duration      22811 non-null  object
 9   listed_in     22998 non-null  object
 10  description   22994 non-null  object
 11  platform      22998 non-null  object
dtypes: int64(1), object(11)
memory usage: 2.1+ MB


Cambiamos el nombre de la columna.

In [35]:
df.rename(columns={'listed_in': 'genre'},inplace=True)

 Normalizamos columna 'duration', sepramos los datos, creamos dos nuevas columnas, dropeamos la columna sobrante y cambiamos el tipo de dato.

In [36]:


df_duracion = df["duration"].str.split(pat= " ", expand= True)

df["duration2"],df["type_duration"] =  df_duracion[0], df_duracion[1]
df.drop(["duration"],axis=1, inplace=True)


In [37]:
df.rename(columns={"duration2":"duration"},inplace=True)
df["duration"] = pd.to_numeric(df["duration"])


Se decide no eliminar las columnas que no se usan, porque no se conoce si puedan tener utilidad en el futuro o quienes mas podrían necesitar de las mismas. Al no tener contacto con el cliente no se puede saber...

Por ultimo, enviamos nuestro Dataframe normalizado a nuestra base de datos.

In [38]:
Database_URL ="sqlite:///../api/Database/database.db"
engine_etl = sql.create_engine(url=Database_URL)

In [39]:
df.to_sql("Database", con = engine_etl, if_exists= "replace", index= True)

22998

In [40]:
df= pd.read_sql('select * from Database',con=engine_etl)


In [41]:
df.columns

Index(['index', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'genre', 'description', 'platform',
       'duration', 'type_duration'],
      dtype='object')

In [42]:
df.iloc[22999]

IndexError: single positional indexer is out-of-bounds